# <center> WSI Ćwiczenie nr4 - Implementacja drzew decyzyjncyh tworzonych algorytmem ID3</center>

### <center> Adam Wróblewski</center>


### Cel eksperymentów:
 Celem eksperymentów jest zbadanie jakości klasyfikatorów dla zbioru danych Cardio Vascular Disease Detection, a także zbadanie wpływu parametru głębokości na wynik.

In [28]:
import pandas as pd
import math
import numpy as np
from sklearn.model_selection import train_test_split

Dyskretyzacja atrybutów wiek (age), waga (weight), wzrost (height), ciśnienie skurczowe (ap_hi), ciśnienie rozkurczowe (ap_lo): 29-65


In [29]:
my_data = pd.read_csv('cardio_train.csv', sep = ';')
# param = 'ap_hi'
# max = my_data.loc[my_data[param].idxmax()]
# min = my_data.loc[my_data[param].idxmin()]
# print('max: ', max[param])
# print('min: ', min[param])
# print(my_data.mean(axis=0))

# dyskretyzacja parametru age 
conditions =[
    my_data['age']/365 <= 40,
    (my_data['age']/365 > 40) & (my_data['age']/365 < 52),
    my_data['age']/365 >= 52 
]
labels =['young', 'middle', 'old']
my_data['age'] = np.select(conditions, labels)

# dyskretyzacja parametru weight
conditions =[
    my_data['weight'] <= 50,
    (my_data['weight'] > 50) & (my_data['weight'] <= 90),
    (my_data['weight'] > 90) & (my_data['weight'] <= 130),
    my_data['weight'] > 130
]
labels =['underweight', 'normal', 'overweight', 'obese']
my_data['weight'] = np.select(conditions, labels)

# dyskretyzacja parametru height
conditions =[
    my_data['height'] <= 140,
    (my_data['height'] > 140) & (my_data['height'] <= 190),
    my_data['height'] > 190
]
labels =['short', 'normal', 'tall']
my_data['height'] = np.select(conditions, labels)

# dyskretyzacja parametru ap_hi
conditions =[
    my_data['ap_hi'] <= 100,
    (my_data['ap_hi'] > 100) & (my_data['ap_hi'] <= 140),
    my_data['ap_hi'] > 140
]
labels =['low', 'normal', 'high']
my_data['ap_hi'] = np.select(conditions, labels)

# dyskretyzacja parametru ap_lo
conditions =[
    my_data['ap_lo'] <= 70,
    (my_data['ap_lo'] > 70) & (my_data['ap_lo'] <= 90),
    my_data['ap_lo'] > 90
]
labels =['low', 'normal', 'high']
my_data['ap_lo'] = np.select(conditions, labels)

Podział danych na zbiór treningowy, walidacyjny, testowy dokonuję przy użyciu biblioteki sklearn.model_selection. Implementacja podziału znajduje się w funkcji "create_train_validate_test_data" 

Implementacja:

In [30]:
class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.output = ""

In [31]:
class Solver():
    """A solver. Parameters may be passed during initialization."""
    def __init__(self, data_as_DataFrame, class_name):
        self.data = data_as_DataFrame
        self.X_data = None
        self.y_data = None 
        self.class_values = None
        self.class_name = class_name
        self.tree = None
        self.train_data = None
        self.validate_data = None
        self.test_data = None

    def create_train_validate_test_data(self):
        data = self.data
        data = data.drop('id', axis=1, inplace=False)
        train_data, rest = train_test_split(data, test_size=0.66, random_state=42)
        validate_data, test_data = train_test_split(rest, test_size=0.5, random_state=42)
        self.train_data = train_data
        self.validate_data = validate_data
        self.test_data = test_data

    def get_data(self, data):
        x_data = data.drop(self.class_name, axis=1, inplace=False)
        y_data = data[self.class_name]
        self.X_data = x_data
        self.y_data = y_data
        self.class_values = sorted(list(set(self.y_data)), reverse=True)
        return x_data, y_data

    def get_parameters(self):
        """Returns a dictionary of hyperparameters"""
        dict = {
            'class_name': self.class_name,
            'class_values': self.class_values
        }
        return dict


    def calculate_entire_entropy(self, class_data):
        class_values = list(set(class_data))
        incydences = []
        for value in class_values:
            incydences.append((class_data.count(value))/len(class_data))

        entire_entropy = 0
        for incydence in incydences:
            entropy = incydence * math.log2(incydence)
            entire_entropy += entropy       
        return -entire_entropy
    
    def calculate_entropy_for_feature(self, data): #data - te wiersze które posiadają wybraną wartość feature-a
        positive_count = 0
        negative_count = 0
        for _, row in data.iterrows():
            if row[self.class_name] == self.class_values[0]:
                positive_count += 1
            else:
                negative_count += 1
        
        if positive_count == 0 or negative_count == 0:
            return 0
        else:
            total = positive_count + negative_count
            pos_incydence = positive_count/(total)
            neg_incydence = negative_count/(total)
            return -(pos_incydence * math.log2(pos_incydence) + neg_incydence * math.log2(neg_incydence))
        
    
    def calculate_info_gain_for_feature(self, data, feature_name):
        feature_values = sorted(set(data[feature_name]))
        rows_count = data.shape[0]
        info_gain = 0
        entire_entropy = self.calculate_entire_entropy(list(data[self.class_name]))
        info_gain += entire_entropy

        for feature_value in feature_values:
            feature_value_data = data[data[feature_name] == feature_value]
            feature_value_incydence = feature_value_data.shape[0] / rows_count
            feature_value_entropy = self.calculate_entropy_for_feature(feature_value_data) 
            info_gain -= feature_value_incydence * feature_value_entropy
        return info_gain


    def find_best_feature(self, data):
        features = list(data.columns.drop(self.class_name))
        best_info_gain = -100000
        best_info_feature = None

        for feature in features:
            feature_info_gain = self.calculate_info_gain_for_feature(data, feature)
            if feature_info_gain > best_info_gain:
                best_info_gain = feature_info_gain
                best_info_feature = feature
        
        return best_info_feature
 

    def ID3(self, data, depth):
        root = Node()
        
        best_feature = self.find_best_feature(data)
        root.value = best_feature
        feature_values = sorted(set(data[best_feature]))
        
        if depth == 0:
            return None 

        for feature_value in feature_values:
            feature_data = data[data[best_feature] == feature_value]
            entropy = self.calculate_entropy_for_feature(feature_data)
            if entropy == 0:
                leaf_node = Node()
                leaf_node.isLeaf = True
                leaf_node.value = feature_value
                leaf_node.output = list(feature_data[self.class_name].unique())[0]
                root.children.append(leaf_node)
            
            else:
                mid_node = Node()
                mid_node.value = feature_value
                stripped_data = feature_data.drop(best_feature, axis=1, inplace=False)
                child = self.ID3(stripped_data, depth - 1)
                if child != None:
                    mid_node.children.append(child)
                elif child == None:
                    mid_node.isLeaf = True
                    mid_node.output = data[self.class_name].value_counts().idxmax()
                root.children.append(mid_node)
        self.tree = root
        return root


    def print_tree(self, root: Node, depth=0):
        for i in range(depth):
            print("\t", end="")
        print(root.value, end="")
        
        if root.isLeaf:
            print(" -> ", root.output)
        print()
        
        for child in root.children:
            self.print_tree(child, depth + 1)
    

    
    def fit(self, X, y, depth):
        """
        A method that fits the solver to the given data.
        X is the dataset without the class attribute.
        y contains the class attribute for each sample from X.
        It may return anything.
        """
        connected_data = pd.concat([X,y], axis=1)
        fitted_tree = self.ID3(connected_data, depth)
        return fitted_tree
        

    def predict(self, X, tree):
            """
            A method that returns predicted class for each row of X
            """
            feature_name = tree.value
            for node in tree.children:
                a = node.value
                b = X[feature_name]
                if node.value == X[feature_name]:
                    if node.isLeaf:
                        # print ("Predicted output for", X ," is:", node.output)
                        return node.output
                    else:
                        result = self.predict(X, node.children[0])
                        if result != '':
                            return result


    def validate(self, tree, my_validate_data):       
        incorrect_predict_count = 0
        correct_predict_count = 0

        for _, row in my_validate_data.iterrows():
            # print(row)
            prediction = self.predict(row, tree)
            real_value = row[self.class_name]
            if prediction == real_value:
                correct_predict_count += 1
            else:
                incorrect_predict_count += 1
        accuracy = correct_predict_count/(correct_predict_count+incorrect_predict_count)
        return accuracy

In [32]:
my_solver = Solver(my_data,'cardio')
my_solver.create_train_validate_test_data()
X, y = my_solver.get_data(my_solver.train_data)
print(my_solver.train_data.shape)

(23562, 12)


In [33]:
tree_depth_1 = my_solver.fit(X,y,1)
tree_depth_2 = my_solver.fit(X,y,2)
tree_depth_3 = my_solver.fit(X,y,3)
tree_depth_4 = my_solver.fit(X,y,4)
tree_depth_5 = my_solver.fit(X,y,5)
tree_depth_6 = my_solver.fit(X,y,6)
tree_depth_7 = my_solver.fit(X,y,7)
tree_depth_8 = my_solver.fit(X,y,8)
tree_depth_9 = my_solver.fit(X,y,9)
tree_depth_10 = my_solver.fit(X,y,10)

In [36]:
score_depth_1 = my_solver.validate(tree_depth_1, my_solver.validate_data)
print('dokładność dla drzewa głębokości 1 i zbioru walidacyjnego = ', score_depth_1)

score_depth_2 = my_solver.validate(tree_depth_2, my_solver.validate_data)
print('dokładność dla drzewa głębokości 2 i zbioru walidacyjnego = ', score_depth_2)

score_depth_3 = my_solver.validate(tree_depth_3, my_solver.validate_data)
print('dokładność dla drzewa głębokości 3 i zbioru walidacyjnego = ', score_depth_3)

score_depth_4 = my_solver.validate(tree_depth_4, my_solver.validate_data)
print('dokładność dla drzewa głębokości 4 i zbioru walidacyjnego = ', score_depth_4)

score_depth_5 = my_solver.validate(tree_depth_5, my_solver.validate_data)
print('dokładność dla drzewa głębokości 5 i zbioru walidacyjnego = ', score_depth_5)

score_depth_6 = my_solver.validate(tree_depth_6, my_solver.validate_data)
print('dokładność dla drzewa głębokości 6 i zbioru walidacyjnego = ', score_depth_6)

score_depth_7 = my_solver.validate(tree_depth_7, my_solver.validate_data)
print('dokładność dla drzewa głębokości 7 i zbioru walidacyjnego = ', score_depth_7)

score_depth_8 = my_solver.validate(tree_depth_8, my_solver.validate_data)
print('dokładność dla drzewa głębokości 8 i zbioru walidacyjnego = ', score_depth_8)

score_depth_9 = my_solver.validate(tree_depth_9, my_solver.validate_data)
print('dokładność dla drzewa głębokości 9 i zbioru walidacyjnego = ', score_depth_9)

score_depth_10 = my_solver.validate(tree_depth_10, my_solver.validate_data)
print('dokładność dla drzewa głębokości 10 i zbioru walidacyjnego = ', score_depth_10)

dokładność dla drzewa głębokości 1 i zbioru walidacyjnego =  0.49857886221522585
dokładność dla drzewa głębokości 2 i zbioru walidacyjnego =  0.6000262363898727
dokładność dla drzewa głębokości 3 i zbioru walidacyjnego =  0.6521929249201976
dokładność dla drzewa głębokości 4 i zbioru walidacyjnego =  0.6561721107175653
dokładność dla drzewa głębokości 5 i zbioru walidacyjnego =  0.6628623901351174
dokładność dla drzewa głębokości 6 i zbioru walidacyjnego =  0.6642179369452097
dokładność dla drzewa głębokości 7 i zbioru walidacyjnego =  0.6682845773754865
dokładność dla drzewa głębokości 8 i zbioru walidacyjnego =  0.6656172110717565
dokładność dla drzewa głębokości 9 i zbioru walidacyjnego =  0.6641304823123005
dokładność dla drzewa głębokości 10 i zbioru walidacyjnego =  0.6599763872491146


Jak widzimy dokładność wyznaczania wyjścia modelu jest najlepsza dla drzewa o głębokości 6. Dla mniejszych wartości głębokości: 3, 4, 5 dokładność jest bardzo zbliżona, natomiast dla głębokości drzewa równych 1 i 2 precyzja jest znacznie mniejsza - przyczyną takiego zjawiska jest duża niedokładność modelu przy tak niskiej głębokości.

Precyzja dla wartości głębogości większej od 7 nieznacznie spada, wynikać to może z nadmiernego dopasowania - przy zbyt dokładnym modelu dane inne niż trenujące powodują błędy.

Z wyżej przedstawionych danych wynika że najlepszym modelem będzie drzewo decyzyjne o głębokości równej 7.

In [37]:
score = my_solver.validate(tree_depth_7, my_solver.test_data)
print('dokładność dla zbioru testowego i drzewa o głębokości 7: ', score)

dokładność dla zbioru testowego i drzewa o głębokości 7:  0.6745080891998251


Dokładność dla zbioru testowego wynosi 0,67. Wynik ten reprezentuje skuteczność modelu.

### Wnioski
Algorytm ID3 służacy do stworzenia modelu jakim są drzewa decyzyjne jest stosunkowo prosty w implementacji.
Skuteczność modelu przez niego generowanego zależy od wielu czynników:<br>
• zbioru danych wejściowych.<br>
• sposobie podziału zbioru danych na zbiory treningowy, walidacyjny, testowy.<br>
• doboru parametrów modelu przy użyciu zbioru walidacyjnego - możliwe jest nadmierne dopasowanie, co spowoduje że dane ze zbioru testowego będą gorzej klasyfikowane, a tym samym błędy modelu będą większe.<br>
• ilości danych wejściowych - im więcej danych tym dokładniejszy model otrzymamy.<br>
• ilości atrybutów zbioru danych - przy bardzo dużej ilości atrybutów wygenerowany model może działać niepoprawnie, lub jego generacja moze trwać bardzo długo.<br><br>


Wpływ głębokości drzewa na poprawność estymacji wyniku:<br>
Generalnie im większa głębokość drzewa tym model jest dokładniejszy - jest o wiele bardziej dopsasowany do danych trenujących i estymata wyjścia jest lepsza. <br>
Jednak przy zbyt dużych głębokościach obserwujemy nieznaczne zmniejszenie precyzji, wynika to z nadmiernego dopasowania modelu do danych trenujących co powoduje wzrost błędów modelu.

Warto zaznaczyć że mimo tego że algorytm nie uwzględnia wszystkich atrybutów w drzewie decyzyjnym to przy bardzo dużej ilości danych proces generowania drzewa decyzyjnego jest długotrwały i czas potrzebny na wykonanie obliczeń znacząco się wydłuża, natomiast na potrzeby walidacji nowych danych wystarczy jednorazowo wygenerować drzewo. 

Zaletą jest także fakt że klasyfikacja nowych danych na podstawie modelu w postaci drzewa decyzyjnego nie wymaga długiego czasu obliczeniowego i przebiega bardzo szybko.
